# Курсовой проект по дисциплине "Численные методы"
# Тема: "Метод бисопряженных градиентов."

## Подготовила студентка группы М8О-307 Алексюнина Юлия
## Руководитель: Ревизников Д.Л.

### Постановка задачи

Реализовать решение систем линейных алгебраических уравнений с 
несимметричными разреженными матрицами большой размерности методом бисопряженных градиентов.


### Описание метода

    Создадим случайную матрицу, поставим плотность (density) = 0.6, таким образом, наша матрица будет на 40% состоять из нулей. Остальные значения будут находиться в диапазоне от 0 до 1. Можно выбрать размер матрицы от 4 до n.

    Рассмотрим СЛАУ с вещественными коэффициентами(Ax=b). В основу алгоритма ложится идея проекционного метода и использование свойства A-бисопряженности системы векторов, а именно: векторы  p – невязки бисопряжённые, если скалярные произведения (Api,pj)=0, (Api,pj)=0. Фактически, данное условие эквивалентно биортогональности относительно энергетического скалярного произведения.
    
    Общая концепция проекционных методов предписывает нам выбрать два подпространства. В нашем случае подпространства мы выберем два подпространства, задаваемые матрицей системы А, вектором невязки на нулевой итерации.

    Метод основан на построении биортогонального базиса p пространства  K_k(A,r0) K_k(A,r0) и вычислении поправки такой, что новое приближение на очередной итерации было бы ортогонально второму подпространству Крылова. Базисные вектора строятся до тех пор, пока не будет достигнут критерий остановки итерационного процесса, а каждое последующее приближение формируется, как сумма приближения на предыдущей итерации и найденной поправки. 
    
    Критерием останова итерационного процесса является достижение невязкой значения, которое меньше некоторого наперед заданного эпсилон.


### Пример теста

![Тест11](https://sun9-68.userapi.com/impf/CH6PryquaKlefjV4ZO4e50egfe9txRQ5Q6mlrw/Jq6Ae8i7uD4.jpg?size=663x484&quality=96&proxy=1&sign=a27eb9cb1438c9179bd2f88689d89c5c&type=album)
![Тест12](https://sun9-14.userapi.com/impf/QNFww4YgWiYQl6twpot8RiAoUTvpNKPcdjz3AQ/UwFWcfN8lOs.jpg?size=485x258&quality=96&proxy=1&sign=da2c866595ca418d161b57a5151c239d&type=album)

### Решение

#### Генерация матрицы. 
Импортируем необходимые библиотеки.


In [6]:
import numpy as np
from random import randint
from scipy.sparse import rand
import numpy as np
from numpy.linalg import norm
from scipy.sparse import diags, csc_matrix
from scipy.sparse.linalg import bicgstab, spilu 

Сгенерируем матрицу заданного размера с плотностью 0.6 и вектор правой части.

In [7]:
shape = int(input())
if shape < 3:
        exit()

matrix = rand(shape, shape, density=0.6, random_state=randint(112, 154))


for i in matrix.toarray().round(3):
    for j in i:
        print(f'{j} ', end=" ")
    print('\n')
print('\n')
b = np.random.randint(5, 53, shape)
for i in b:
    print(f'{i} ', end=" ")
print('\n')
print(type(matrix))

16
0.449  0.606  0.22  0.418  0.0  0.0  0.0  0.0  0.327  0.304  0.53  0.0  0.006  0.493  0.182  0.0  

0.0  0.0  0.198  0.208  0.0  0.0  0.928  0.0  0.0  0.002  0.0  0.37  0.0  0.0  0.762  0.0  

0.0  0.0  0.87  0.0  0.382  0.806  0.961  0.348  0.0  0.416  0.0  0.0  0.0  0.868  0.412  0.983  

0.928  0.962  0.0  0.0  0.202  0.0  0.0  0.448  0.307  0.0  0.0  0.0  0.0  0.396  0.0  0.265  

0.0  0.971  0.731  0.0  0.752  0.586  0.866  0.0  0.861  0.0  0.847  0.134  0.243  0.0  0.065  0.0  

0.389  0.64  0.819  0.0  0.0  0.964  0.94  0.724  0.618  0.077  0.0  0.9  0.74  0.782  0.0  0.0  

0.7  0.54  0.063  0.821  0.052  0.96  0.853  0.0  0.0  0.0  0.0  0.247  0.0  0.399  0.213  0.49  

0.91  0.986  0.791  0.859  0.754  0.093  0.077  0.0  0.447  0.214  0.0  0.0  0.218  0.153  0.69  0.0  

0.0  0.0  0.381  0.0  0.0  0.967  1.0  0.61  0.574  0.0  0.639  0.0  0.0  0.443  0.748  0.576  

0.0  0.0  0.953  0.59  0.32  0.384  0.0  0.0  0.805  0.951  0.0  0.67  0.616  0.741  0.0  0.306  

0.927  0.

#### Реализация алгоритма

![Алгоритм](https://sun9-31.userapi.com/impf/eB8P4H6U7fR28U9YD9XuwvE1emlksS9FQhuEfg/zalHp3WbPW0.jpg?size=370x441&quality=96&proxy=1&sign=dc07ec63bc662226176c3a6a42efafc0)

Критерий остановки итерационного процесса: норма заданной невязки меньше эпсилон.

Представим исходную матрицу в виде сжатой матрицы разреженных столбцов.

In [8]:
matrix = csc_matrix(matrix)

Создадим класс BiCGMethod, в котором реализуем несколько методов. 
    __init__ - для инициализации необходимых данных;
    solve - подготовка перед итерационным процессом и сам процесс решения;
    print_solution - печать результата и сравнение его с решением при помощи функций встроенной библиотеки Numpy

In [9]:
class BiCGMethod:
    def __init__(self, matrix, b, x0=None, eps=1e-5):
        self.matrix = matrix
        self.b = b
        self.eps = eps
        self.shape = matrix.shape[0]
        self.x0 = np.array([0] * self.shape) if x0 is None else x0
        self.k = 0
        
    def solve(self):
        r0 = self.b - self.matrix @ self.x0 # невязка
        x0 = self.x0 # начальное приближение
        r2 = r0 # выбирается вектор
        rho0 = 1
        alpha0 = 1
        omega0 = 1
        v0 = np.array([0] * self.shape)
        p0 = np.array([0] * self.shape)
        while True:
            rho = r2 @ r0
            beta = (rho * alpha0) / (rho0 * omega0)
            p = r0 + beta * (p0 - omega0 * v0)
            v = self.matrix @ p
            alpha = rho / (r2 @ v)
            s = r0 - alpha * v
            t = self.matrix @ s
            omega = (t @ s) / (t @ t)
            x = x0 + omega * s + alpha * p
            r = s - omega * t


            self.k += 1
            if norm(r) < self.eps: # норма заданной невязки
                break
            r0 = r
            rho0 = rho
            alpha0 = alpha
            omega0 = omega
            v0 = v
            p0 = p
            x0 = x
        return x
    
    def print_solution(self):
        x = self.solve()
        x2 = np.linalg.solve(self.matrix.toarray(), self.b)
        # with open(self.output, 'w') as f:
        print('My solve:\n')
        print(f'{x.round(5)}\n')
        print(f'EPS = {self.eps}\n')
        print(f'Shape = {self.shape}\n')
        print(f'Count of iterations = {self.k}\n')
        # print(f'Mean = {np.mean(x)}\n') # среднее
        print('\nNumPy solve:\n')
        print(f'{x2.round(5)}\n')
        # print(f'Mean = {np.mean(x2)}\n')


Наконец, сделаем расчет.

In [10]:
solver = BiCGMethod(matrix, b, eps=1e-5)
solver.print_solution()

My solve:

[ 1004.38337  -717.99638   611.83009  -160.23214    83.38022   287.43935
   347.21342   561.81247   106.93063  1022.62429    87.64077   263.69304
 -1724.79793  -674.58601  -631.29276  -912.37142]

EPS = 1e-05

Shape = 16

Count of iterations = 35


NumPy solve:

[ 1004.38336  -717.99638   611.83009  -160.23213    83.38022   287.43935
   347.21342   561.81247   106.93063  1022.62427    87.64077   263.69303
 -1724.79792  -674.58601  -631.29276  -912.37141]



### Вывод
В ходе данной лабораторной работы был изучен алгоритм бисопряженных градиентов. Этот алгоритм хорошо зарекомендовал себя для решения систем линейных алгебраический уравнений, поскольку обладает некоторыми важными свойствами: он численно устойчив; не меняется вид матрицы в процессе решения; эффективен для решения систем большой размерности с разреженной матрицей, поскольку в методе самая трудоемкая операция - умножение матрицы на вектор; применим для решения систем с несимметричными матрицами. Вычислительная мощность:  N/4